<a href="https://colab.research.google.com/github/EmoChat/EmoChat/blob/master/EmoChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import *

In [5]:
import matplotlib.pylab as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from keras.preprocessing import image
import pathlib
import pandas as pd

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [40]:
batch_size = 32
img_height = 250
img_width = 250
IMG_SHAPE = (img_height, img_width, 3)
class_names = ["Disgusted", "Happy", "Neutral", "Angry", "Surprise"]

In [ ]:
train = pd.read_csv('train.csv', names=['File_name', 'Label'])
train_image = [ ]

for i in tqdm(range(train.shape[0])):

  img = image.load_img('Images/train/'+train['File_name'][i], target_size=(28,28,1), 
              grayscale=False)
  img = image.img_to_array(img)
  img = img/255
  train_image.append(img)

X = np.array(train_image)

train_labels = train.Label.to_list()

In [36]:
VGG_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
VGG_model.trainable = False

In [37]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.CenterCrop(img_height, img_width))
model.add(VGG_model)
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [38]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x=feature,
                      y=label,
                      batch_size=batch_size,
                      epochs=epochs)